# Physics-based Sound Synthesis

## Motivation
![taxonomy_sound_synthesis](images/taxonomy_sound_synthesis.png)
There are four approaches for sound synthesis. Traditionally, 

### Karplus-Strong algorithm
Karplus-Strong algorithm is an abstract algorithmic sound synthesis method.
![karplus_strong](images/karplus_strong.png)
**It's a self-modifying wavetable.** Wavetable is interpreted as a delay line, where the waveform propagates. Dullness is reduced considerably. Timbre and decay time depend on sampling rate



#### Tuning problem

High notes are **out of tune**: 
- $f=f_{S} /(L+0.5)$, where the half-sample delay is caused by averaging.
- Wavetable length $L$ is rounded to the nearest integer

![karplus_strong_tuning_problem](images/karplus_strong_tuning_problem.png)



## Introduction of Sound Synthesis by Physical Modeling
- Simulate the sound production mechanism
    - In speech synthesis and coding, this is an old tradition
- High-quality synthesis of string and wind instruments
- **More sophisticated signal processing than earlier**
    - Compared to subtractive synthesis, sampling & wavetable synthesis, FM synthesis
    - Combines linear & nonlinear methods, filtering, sampling
    - Considered computationally expensive

## Calssification of methods

1. Source-filter modeling
    - Physically informed subtractive synthesis
    - It's like an emulation of animal speech: the source signal is processed by a filter, e.g. a formant filter, and the resulting signal has a resonating structure such as a vocal tract or soundbox of an instrument. 
2. Finite Difference Modeling
    - Discretized wave equation (difference equation)
3. Mass-Spring Networks
    - "CORDIS-ANIMA" by ACROE (Grenoble, France)
4. Modal Synthesis
    - "Modalys" software by IRCAM; formerly "Mosaic"
    - The functional transform method by Trautmann and Rabenstein
5. Wave Digital Filters
    - Method to convert models of continuous-time systems into digital filters
6. Digital Waveguide Synthesis
    - Physical modeling using digital filters and delays; Main topic of this lecture

## Digital waveguide modeling

### Tuning with Fractional delay filter
Fine-tunes the delay-line length so that the pitch is right

### Tuning with Allpass filter
First-order allpass filter usually. Transient problems in time-varying cases.
- Phase delay of the first-order allpass filter at low frequencies(dc): $D=\left(1-a_{1}\right) /\left(1+a_{1}\right)$
- Phase delay: $-\phi(\omega) / \omega$
    - Phase response divided by angular frequency
- Solve $a_1$ for desired delay $D$: $a_{1}=\frac{1-D}{1+D}$

![tuning_all_pass](images/tuning_all_pass.png)

### Tuning with Lagrange interpolation
Order 3 or 4 is sufficient at 44.1 kHz. Linear interpolation(1st-order Lagrange) is not good enough because high frequencies will be damped.

Example: 3rd-order Lagrange FD filter ($N=3$), which is a four-tap FIR filter:
![tuning_with_lagrange_interpolation](images/tuning_with_lagrange_interpolation.png)

Coefficient formulas for 3rd-order Lagrange FD filter ($1 < D<2$):
$$
\begin{align}
    h(0) &=-\frac{1}{6}(D-1)(D-2)(D-3) \\
    h(1) &=\frac{1}{2} D(D-2)(D-3) \\
    h(2) &=-\frac{1}{2} D(D-1)(D-3) \\
    h(3) &=\frac{1}{6} D(D-1)(D-2)
\end{align}
$$

#### Loop filter
- Loop filter $H_l(z)$ controls the decay rate of harmonics
- One choice is a **one-pole filter**("leaky integrator"): $H_{l}(z)=g \frac{1+a_{1}}{1+a_{1} z^{-1}}$

    - Parameter $g$ determines overall decay rate ($0 < g < 1$)
    - Parameter $a_1$ determines frequency-dependent decay ($-1 < a_1 < 0$)

#### 

#### 

## Modeling of stringed musical instruments
Stringed instruments include both string and keyboard instruments such as guitar, harpsichord, piano, Clavinet and etc.

## Conclusions

- Several methods available for physics-based sound synthesis
    - Source-filter, modal, mass-spring, FDTD, wave digital and digital waveguide models
- Digital waveguide synthesis used to be the most popular method 
- Modal synthesis and FDTD methods have become more attractive thanks to faster computers